# Django - Wprowadzenie 2
*[Mikołaj Leszczuk](mailto:mikolaj.leszczuk@agh.edu.pl), [Agnieszka Rudnicka](mailto:rudnicka@agh.edu.pl)*

* Nasza pierwsza aplikacja
* Nasz pierwszy widok i automatyczne przeładowanie aplikacji
* Nasz pierwszy "pełnoprawny" widok
* Bazy danych i migracja bazy danych
* Tworzenie super-użytkownika
* Panel administracyjny
* Kontekst w szablonach HTML
* Jak włączyć obsługę Django w PyCharm?
* Zadanie

## Nasza pierwsza aplikacja

W środowisku Django przyjęło się, że każdy projekt zrzesza wiele mniejszych aplikacji (Django apps). Rolą aplikacji jest zapewnienie konkretnej funkcjonalności, w miarę możliwości niezależnej od innych. Można na to patrzeć jak na swego rodzaju *pluginy*. Takie appki w idealnym świecie można by wydzielać i przenosić między projektami.

Stwórzmy więc pierwsza aplikację, na cele tego kursu będziemy tworzyć mini-bibliotekę zawierającą filmy.

In [ ]:
!django-admin startapp movies

Powyższe polecenie stworzy nowy katalog z kilkoma plikami. Omówimy je poniżej.

In [ ]:
!tree

* *migrations/* - katalog na migracje bazodanowe, czyli pliki, które opisują jak schemat bazy danych się zmieniał pomiędzy kolejnymi wersjami aplikacji. Tutaj znajdziemy wyłącznie pliki automatycznie generowane przez Django. W praktyce - niekiedy zachodzi potrzeba napisania migracji danych i to tutaj się je tworzy (poza zakresem tego kursu).

* *admin.py* - plik, w którym deklarujemy jak ma wyglądać nasz panel administracyjny, w praktyce opisuje się tutaj które modele i jakie pola modeli mają być edytowalne.

* *apps.py* - plik deklarujący podstawowe informacje o aplikacji, takie jak nazwa "dla ludzi"

* *models.py* - jeden z ważniejszych plików, to tutaj znajdują się informacje jak wyglądają dane przechowywane w bazie danych przez aplikacji. W naszym przypadku będą to filmy, więc tutaj znajdziemy model opisujący pojedynczy film (spis pól jak tytuł, reżyser, data premiery, aktorzy...)

* *tests.py* - tak, testy :)

* *views.py* - wspomniane wcześniej *widoki* zamieszczane są w tym pliku. Dzięki logice tutaj umieszczonej Django będzie w stanie wyświetlić nasze strony internetowe.

## Nasz pierwszy widok i automatyczne przeładowanie aplikacji

Otwórzmy plik [`views.py`](http://localhost:8888/edit/movies/views.py) umieszczony w [`movies/views.py`](http://localhost:8888/edit/movies/views.py).

Na początek posłużymy się prostą funkcją, która jako argument przyjmuje zapytanie (które przyjdzie z naszej przeglądarki). Funkcja ta powinna jakoś odpowiedzieć.

Na górze pliku zaimportujmy generyczną odpowiedź HTTP:

```python
from django.http import HttpResponse
```

Następnie napiszmy prostą funkcję, która zwraca jedynie napis "Witaj świecie":

```python
def hello_world(request):
    return HttpResponse("Witaj świecie!")
```

Nasz plik [`views.py`](http://localhost:8888/edit/movies/views.py) powinien wyglądać następująco:

```python
from django.http import HttpResponse
from django.shortcuts import render


# Create your views here.
def hello_world(request):
    return HttpResponse("Witaj świecie!")
```

Jednak to nie wszystko. Jeśli wejdziemy [http://127.0.0.1:8000/](http://127.0.0.1:8000/) nie ukaże się nam powitanie świata.

Aby nasza aplikacja była uwzględniona przez Django trzeba ją dodać do zainstalowanych. W tym celu otwieramy plik [`settings.py`](http://localhost:8888/edit/goodmovies/settings.py), o którym była mowa wcześniej. To tutaj znajdują się ustawienia projektu, w tym lista zainstalowanych aplikacji.

Odszukujemy fragment z listą o nazwie `INSTALLED_APPS`:

```python
# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'movies',  # <-- NOWE, dopisane teraz :)
]
```

I dopisujemy naszą aplikację, w moim przypadku to jest `movies`.

W tej chwili nasz aplikacja została dołączona do projektu. Jednakże widok, który napisaliśmy nie został podpięty pod żaden adres URL, więc nie da się go jeszcze wyświetlić. Naprawmy to.

Otwórzmy [`urls.py`](http://localhost:8888/edit/goodmovies/urls.py) w katalogu projektu i dodajmy naszą funkcję widoku:

```python
from movies import views  # NOWE

urlpatterns = [
    path('admin/', admin.site.urls),
    path('hello/', views.hello_world),  # NOWE
]
```

W powyższym kodzie `# NOWE` zaznaczono 2 dodane linijki. Reszta powinna zostać bez zmian.

Następnie udajmy się do przeglądarki pod adres [http://127.0.0.1:8000/hello/](http://127.0.0.1:8000/hello/)

Działa! 😊

W międzyczasie można było zauważyć (jeśli aplikacja była uruchomiona), ze doszło do automatycznego przeładowania aplikacji:

```sh
.../views.py changed, reloading.
Performing system checks...

Watching for file changes with StatReloader
```

Jest to mechanizm automatycznego przeładowania, który jest domyślnie włączony i "nasłuchuje" na zmiany w plikach projektu i aplikacji.

Dzięki temu nie musimy ręcznie zatrzymywać i uruchamiać serwera od nowa po każdej wprowadzonej zmianie. Prawda, że przydatne?

## Nasz pierwszy "pełnoprawny" widok

Ten poprzedni pierwszy widok był tylko nudnym tekstem, spróbujmy tym razem użyć odpowiedzi HTML.

Wróćmy do [`views.py`](http://localhost:8888/edit/movies/views.py):

```python
from django.shortcuts import render


# Create your views here.
def hello_world(request):
    return render(request, template_name="hello.html")  # NOWE
```

Tym razem użyjmy funkcji `render`, która już była wcześniej zaimportowana. Przyjmuje ona 2 argumenty wymagane:

* _request_ - czyli przychodzące zapytanie o stronę

* _template_name_ - nazwę szablonu HTML, który ma być wyrenderowany jako odpowiedź

Oprócz tego można też przekazać kontekst, ale do tego wrócimy za chwilę.

Oczywiście plik `hello.html` nie istnieje, więc musimy go wpierw utworzyć.

W katalogu naszej aplikacji [`movies`](movies/) stwórzmy katalog [`templates/`](movies/), to tutaj będziemy umieszczać pliki HTML dotyczące aplikacji [`movies`](movies/).

In [ ]:
!mkdir movies/templates

W tym katalogu utwórzmy plik [`hello.html`](http://localhost:8888/edit/movies/templates/hello.html).

In [ ]:
!touch movies/templates/hello.html

I zamieśćmy w pliku [`hello.html`](http://localhost:8888/edit/movies/templates/hello.html) dowolny HTML, na przykład:<br>[`<h1>Witaj świecie</h1>`](http://localhost:8888/edit/movies/templates/hello.html)

In [ ]:
!echo "<h1>Witaj świecie</h1>" > movies/templates/hello.html

In [ ]:
!tree movies/

Następnie [uruchamiamy ponownie serwer](0_Run.ipynb) (tak, tym razem trzeba!).

I odświeżmy stronę w przeglądarce ([http://127.0.0.1:8000/hello/](http://127.0.0.1:8000/hello/) jeśli ktoś zamknął kartę).

Powinniśmy ujrzeć napis, ale tym razem jako nagłówek `<h1>`.

## Bazy danych i migracja bazy danych

Wróćmy na chwilę do pliku [`settings.py`](http://localhost:8888/edit/goodmovies/settings.py). To tutaj znajdują się kluczowe ustawienia projektu.

Wśród nich jest również informacja o bazie danych:

```python
# Database
# https://docs.djangoproject.com/en/3.0/ref/settings/#databases

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}
```

Jak może zauważyliście, po uruchomieniu projektu w katalogu głównym pojawił się plik `db.sqlite3`. To właśnie nasza baza danych. Django domyślnie przechowuje dane w pliku `'db.sqlite3'` w formacie `SQLite3`.

Gdybyśmy w przyszłości chcieli używać innego silnika bazy danych, trzeba będzie zmienić powyższe ustawienie.

Jednak na potrzeby tego projektu zostaniemy przy SQLite - jest proste w obsłudze i nie wymaga instalowania dodatkowego oprogramowania. Django wspiera jednak również inne serwery baz danych, jak na przykład: PostgreSQL i MySQL.

Podczas uruchomiania projektu można było zauważyć w terminalu poniższą wiadomość:

```sh
System check identified no issues (0 silenced).

You have 18 unapplied migration(s). Your project may not work properly until you apply the
migrations for app(s): admin, auth, contenttypes, sessions.
Run 'python manage.py migrate' to apply them.
```

Django informuje nas, że mamy niezaaplikowane migracje. W skrócie oznacza to, że stan naszej bazy danych nie odzwierciedla tego, czego oczekuje aplikacja. Przykładowo aplikacja `django.contrib.auth` dostarcza model użytkownika, którego użyjemy za chwilę.

Model użytkownika ma takie pola jak _username_, _imię_, _nazwisko_, _hasło_. Django oczekuje, że tak zdefiniowany model będzie miał odpowiednią tabele w bazie danych z kolumnami odpowiedniego typu (na dane tekstowe - tekstowe, na liczby - kolumnę typu liczbowego, na pliki inny itp.).

Żeby lepiej zrozumieć konsekwencje spróbujmy wejść pod adres [http://127.0.0.1:8000/admin/](http://127.0.0.1:8000/admin/)

Zobaczymy formularz logowania. Jednak nie mamy przecież ani loginu, ani hasła potrzebnych do przejścia dalej. Te informacje byłyby zapisane w bazie danych.

![](https://docs.djangoproject.com/en/1.8/_images/admin01.png)

Zmigrujmy zatem naszą bazę danych (tym samym stworzone zostaną tabele na użytkowników i nie tylko).

Wszelkie akcje związane z zarządzaniem aplikacją można wykonać używając skryptu `manage.py` (wcześniejsze tworzenie appki też):

In [ ]:
!python3 manage.py migrate

Podczas [kolejnego uruchomiania projektu](0_Run.ipynb) już nie zauważymy w terminalu wiadomości, w której Django informuje nas, że mamy niezaaplikowane migracje.

## Tworzenie super-użytkownika

Aby stworzyć super-użytkownika, który będzie miał wszystkie uprawnienia w aplikacji możemy użyć polecenia pomocniczego...

In [ ]:
import requests
requests.post('http://127.0.0.1:8888/api/terminals')

...[w terminalu](/terminals/1):

```sh
python3 manage.py createsuperuser
```

Teraz będzie można się zalogować do panelu administracyjnego.

## Panel administracyjny

Przejdźmy więc na stronę: [http://127.0.0.1:8000/admin/](http://127.0.0.1:8000/admin/)

Można tu miedzy innymi zarządzać użytkownikami (klikając w `Users` przejdziemy do listy użytkowników).

Wszystkie znajdujące się tutaj widoki są automatycznie generowane. Na następnych zajęciach zajmiemy się dodawaniem modeli i podepniemy je pod ten panel.

Wylogujmy się.

## Kontekst w szablonach HTML

Wróćmy na chwilę do naszego przykładu z "Witaj świecie" w HTML. Gdybyśmy chcieli wyświetlić informacje o aktualnie zalogowanym użytkowniku możemy umieścić w [`hello.html`](http://localhost:8888/edit/movies/templates/hello.html) taki oto kod:

```django
<p>
    Aktualny użytkownik to: {{ request.user }}
</p>
```

```sh
nano movies/templates/hello.html
```

W przeglądarce powinniśmy zobaczyć:

```
Aktualny użytkownik to: AnonymousUser
```

lub jeśli się zalogujemy przez panel admina - login naszego użytkownika zamiast `AnonymousUser`.

Generalnie za pomocą `{{ zmienna }}` podwójnych nawiasów klamrowych możemy wypisywać dane dostępne w kontekście HTML. Co to znaczy? Myślmy o kontekście jak o słowniku, czyli strukturze klucz-wartość. Pod każdym kluczem kryją się jakieś dane. Wiele z nich jest zapewnionych domyślnie przez framework, jak np. wyżej wykorzystane `request.user`.

Jednak to nie wszystko. Załóżmy, że chcemy dodać do kontekstu własne dane, np. aktualną godzinę.

Wróćmy zatem do naszej funkcji widoku w [`views.py`](http://localhost:8888/edit/movies/views.py):

```python
from datetime import datetime  # NOWE

from django.shortcuts import render


# Create your views here.
def hello_world(request):
    our_context = {"time": datetime.now()}  # NOWE
    return render(
        request, 
        template_name="hello.html", 
        context=our_context
    )  # NOWE
```

W powyższym kodzie stworzyliśmy słownik `our_context`, w którym umieściliśmy aktualną datę i godzinę. Następnie przekazaliśmy ten słownik jako `context` do funkcji `render()`.

Efekt?

W [`hello.html`](http://localhost:8888/edit/movies/templates/hello.html) możemy teraz wypisać zmienną `{{ time }}` używając podwójnych nawiasów klamrowych.

```django
<p>
    Aktualny czas: {{ time }}
</p>
```

```sh
nano movies/templates/hello.html
```

Efekt?

To właśnie przez kontekst będziemy przekazywać informacje z tak zwanego "backendu" do "frontendu". Będą to np. zapisane w naszej bazie danych filmy i inne informacje.

## Jak włączyć obsługę Django w PyCharm?

![](pycharm.png)

## Zadanie

Sprawdź, czy użytkownik jest uwierzytelniony.

Można wykorzystać informacje o użytkowniku z

`{{ user.is_authenticated }}`.